In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
195,AFG,Asia,Afghanistan,2020-07-23,35915.0,188.0,1211.0,21.0,922.593,4.829,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
332,ALB,Europe,Albania,2020-07-23,4358.0,68.0,120.0,3.0,1514.351,23.629,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
533,DZA,Africa,Algeria,2020-07-23,24872.0,594.0,1111.0,11.0,567.193,13.546,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-22,France,179113.0,0.0,30179.0,0.0,0.0,0.0
0,2020-07-22,China,85857.0,0.0,4656.0,0.0,0.0,0.0
0,2020-07-22,Italy,245070.0,0.0,35108.0,0.0,0.0,0.0
0,2020-07-22,Spain,270246.0,0.0,28437.0,0.0,0.0,0.0
0,2020-07-22,United States,3961300.0,0.0,142634.0,0.0,0.0,0.0
0,2020-07-22,World,15207555.0,0.0,621888.0,0.0,0.0,0.0
0,2020-07-22,United Kingdom,296706.0,0.0,45493.0,0.0,0.0,0.0
0,2020-07-22,Germany,203367.0,0.0,9107.0,0.0,0.0,0.0
0,2020-07-22,Iran,282178.0,0.0,14818.0,0.0,0.0,0.0
0,2020-07-22,Turkey,222394.0,0.0,5542.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-23,France,179113.0,178336.0,30179.0,30172.0,777.0,7.0
0,2020-07-23,China,85857.0,85906.0,4656.0,4648.0,-49.0,8.0
0,2020-07-23,Italy,245070.0,245032.0,35108.0,35082.0,38.0,26.0
0,2020-07-22,Spain,270246.0,267551.0,28437.0,28426.0,2695.0,11.0
0,2020-07-23,United States,3961300.0,3970906.0,142634.0,143190.0,-9606.0,-556.0
0,2020-07-23,World,15207555.0,15201538.0,621888.0,623282.0,6017.0,-1394.0
0,2020-07-23,United Kingdom,296706.0,296377.0,45493.0,45501.0,329.0,-8.0
0,2020-07-23,Germany,203367.0,203368.0,9107.0,9101.0,-1.0,6.0
0,2020-07-23,Iran,282178.0,281413.0,14818.0,14853.0,765.0,-35.0
0,2020-07-23,Turkey,222394.0,222402.0,5542.0,5545.0,-8.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")